<a href="https://colab.research.google.com/github/aaalexlit/tf-advanced-techniques-spec/blob/main/course_2_custom_distributed_training/Week4_Mirrored_Strategy_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [12]:
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

## Load mnist

In [13]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

## Define the strategy

In [14]:
strategy = tf.distribute.MirroredStrategy()
print(f'Number of devices: {strategy.num_replicas_in_sync}')

Number of devices: 1


## Define Batch Size

In [15]:
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

## Prepare training and validation set

In [16]:
# Function for normalizing the image
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

In [17]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## Use the strategy

In [18]:
with strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

In [19]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model.fit(train_dataset, epochs=12)

Epoch 1/12
938/938 [==============================] - 11s 9ms/step - loss: 0.1925 - accuracy: 0.9443
Epoch 2/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0625 - accuracy: 0.9816
Epoch 3/12
938/938 [==============================] - 4s 5ms/step - loss: 0.0441 - accuracy: 0.9866
Epoch 4/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0327 - accuracy: 0.9899
Epoch 5/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0252 - accuracy: 0.9919
Epoch 6/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0195 - accuracy: 0.9941
Epoch 7/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0147 - accuracy: 0.9954
Epoch 8/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0113 - accuracy: 0.9967
Epoch 9/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0094 - accuracy: 0.9970
Epoch 10/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0074 - accuracy: 0.997